# 4 自动求导 autograd

## torch.autograd.backward()
函数原型：torch.autograd.backward(tensors, grad_tensors=None, retain_graph=None, create_graph=False, grad_variables=None)
<br>功能：自动求取梯度
<br>tensors: 用于求导的张量，如 loss
<br>retain_graph: 保存计算图。PyTorch 采用动态图机制，**默认每次反向传播之后都会释放计算图。这里设置为 True 可以不释放计算图。**
<br>create_graph: 创建导数计算图，用于高阶求导。
<br>grad_tensors: 多梯度权重。当有多个 loss 混合需要计算梯度时，设置每个 loss 的权重。

#### 解释 retain_graph
在 tensor.backward(retain_graph=True) 的情况下，**第一次求导后保存计算图，可以再次进行求导。否则计算图会被释放，无法再次求导。**

In [6]:
import torch
w = torch.tensor([1], requires_grad=True, dtype=float)
x = torch.tensor([2], requires_grad=True, dtype=float)
a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

# y.backward()
# print("gradient:", w.grad)
# y.backward()      # RuntimeError: 试图对计算图第二次调用 backward()，当调用.backward()或autograd.grad()时，计算图中保存的导数中间值将被释放。
                    # 如果需要对计算图二次求导，指定 retain_graph=True
y.backward(retain_graph=True)
print("First gradient:", w.grad)
y.backward()
print("Second gradient:", w.grad)
# y.backward()      # 要想下一次计算图可以再次自动求导，那么这次求导时参数就需要指定 retain_graph=True

gradient: tensor([5.], dtype=torch.float64)
gradient: tensor([10.], dtype=torch.float64)


#### 解释 grad_tensors
多个损失函数，多权重求导

In [8]:
w = torch.tensor([1], requires_grad=True, dtype=float)
x = torch.tensor([2], requires_grad=True, dtype=float)
a = torch.add(w, x)
b = torch.add(w, 1)

y0 = torch.mul(a, b)
y1 = torch.add(a, b)

loss = torch.cat((y0, y1), dim=0)   # 将两个函数拼接在一起 2*1维
grad_tensor = torch.tensor([1, 2])     # 设置两个 loss 的权重: y0 的权重是 1，y1 的权重是 2

loss.backward(gradient=grad_tensor)    # 最终的 w 的导数由两部分组成。∂y0/∂w * 1 + ∂y1/∂w * 2
print(w.grad)

tensor([9.], dtype=torch.float64)


该 loss 由两部分组成：$y{0}$ 和 $y{1}$。其中 $\frac{\partial y{0}}{\partial w}=5$，$\frac{\partial y{1}}{\partial w}=2$。而 gradtensor 设置两个 loss 对 w 的权重分别为 1 和 2。因此最终 w 的梯度为：$\frac{\partial y_{0}}{\partial w} \times 1+ \frac{\partial y_{1}}{\partial w} \times 2=9$。

## torch.autograd.grad()
函数原型：torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False, only_inputs=True, allow_unused=False)
<br>功能：求取梯度
<br>outputs: 用于求导的张量，如 loss
<br>inputs: 需要梯度的张量，如 w
<br>create_graph: 创建导数计算图，用于高阶求导。
<br>retain_graph:保存计算图，可以再次求导
<br>grad_tensors: 多梯度权重。当有多个 loss 混合需要计算梯度时，设置每个 loss 的权重。

In [11]:
# 计算高阶导
x = torch.tensor([2], dtype=float, requires_grad=True)
y = torch.pow(x, 2)     # y = x**2
gradFirst = torch.autograd.grad(y, x, create_graph=True)    # 求一阶导，指定参数 create_graph=True 是为了让一阶导数 grad_1 也拥有计算图，利用此计算图求二阶导
print(gradFirst)                                            # 返回的是一个元组，第 0 个元素才是真正的梯度
gradSecond = torch.autograd.grad(gradFirst[0], x)
print(gradSecond)

(tensor([4.], dtype=torch.float64, grad_fn=<MulBackward0>),)
(tensor([2.], dtype=torch.float64),)


### 注意

#### **在每次反向传播求导时，计算的梯度不会自动清零。如果进行多次迭代计算梯度而没有清零，那么梯度会在前一次的基础上叠加。**

In [17]:
w = torch.tensor([2.], requires_grad=True)
x = torch.tensor([1.], requires_grad=True)
print("不清零状态：")
for i in range(0, 4):
    a = torch.add(w, x)
    b = torch.add(w, 1)
    y = torch.mul(a, b)
    y.backward()
    print(w.grad)

# 使用w.grad.zero_()将梯度清零。
print("清零状态：")
for i in range(0, 4):
    a = torch.add(w, x)
    b = torch.add(w, 1)
    y = torch.mul(a, b)
    w.grad.zero_()      # 清零
    y.backward()
    print(w.grad)

不清零状态：
tensor([6.])
tensor([12.])
tensor([18.])
tensor([24.])
清零状态：
tensor([6.])
tensor([6.])
tensor([6.])
tensor([6.])


#### inplace操作 未读 TODO！！